In [ ]:
import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as ppp
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean

#Necessary for the map plots

ppp.plotly.mapbox_plot.set_mapbox_token('pk.eyJ1IjoiYTA1MTIxOSIsImEiOiJjazNzanQxNWcwNzlmM2hvNHI0azJmZXJxIn0.Ml2ah_liMPO_zPyrXEpoVw')

In [ ]:
'''
This cell re-initialises the network

There are two options available:
1. Re-initialise from the original GBreducednetwork
2. Re-initialise from the modified network stored in, net_geodata.xlsx

net_geodata.xlsx contains the real-world geodata stored in lat/long for the buses
It also contains updated demand and generation data to better reflect changes from 2019.

The network is outputted to a readable Excel file in the current working directory.
'''

net = pp.from_excel('net_geodata.xlsx')

scaling = 1.0
net.load.scaling = scaling

#net = pn.GBreducednetwork()
pp.runpp(net)

pp.to_excel(net,'net.xlsx')

print(f"Mean Loading Percent = {mean(net.res_line['loading_percent'])}")
print(f"Cumulative Loading Percent = {sum(net.res_line['loading_percent'])}")
print(f"Maximum Loading Percent = {max(net.res_line['loading_percent'])}")

In [ ]:
'''
This cell drops every individual line in sequence, either in a defined list or in the full dataset
Generates a summarised results table, 'Results.xlsx'

'Results.xlsx' contains:
The MEAN of all remaining loading percentages
The SUM of all loading percentages
The MAX loading percentage in the remaining network
'''

#net = pn.GBreducednetwork()
net = pp.from_excel('net_geodata.xlsx')
net.load.scaling = scaling
print(f"Running {len(net.res_line)} lines...")

index = []
mean_lp = []
sum_lp = []
max_lp = []

#Enable for running the dataset on a specific sub-set of lines:
#spec_bus = []

for i in range(0,len(net.res_line)):
#for i in range(70,78):
#for i in spec_bus:
    pp.drop_lines(net,[i])
    index.append(i)
    try:
        pp.runpp(net, max_iteration=200)
        mean_lp.append(mean(net.res_line['loading_percent']))
        sum_lp.append(sum(net.res_line['loading_percent']))
        max_lp.append(max(net.res_line['loading_percent']))
        print(i, end=' - ')
    except:
        print(f"({i})", end=' - ')
        mean_lp.append("Did not converge.")
        sum_lp.append("Did not converge.")
        max_lp.append("Did not converge.")
#    net = pn.GBreducednetwork()
    net = pp.from_excel('net_geodata.xlsx')
    net.load.scaling = scaling
    
print("END\nOperation complete. Results dumped to Results.xlsx.")
    
data = {'Line': index,
      'Mean Loading Percent': mean_lp,
      'Sum Loading Percent': sum_lp,
      'Maximum Loading Percent': max_lp }
 
df = pd.DataFrame(data)

df.to_excel('Result.xlsx')

In [ ]:
'''
This cell drops a specified individual line and runs the full network when that line is dropped.
Network is outputted to an Excel file in the current working directly.
You can also then plot maps etc. with the dropped network.
You can also repeat this cell to drop additional lines.

For a full network analysis, use the above cell, which does the same thing iteratively for every line in sequence.
'''

#net = pn.GBreducednetwork()
#net = pp.from_excel('net_geodata.xlsx')

x = int(input("Enter line to drop > "))

pp.drop_lines(net,[x])

pp.runpp(net, max_iteration=1000)

pp.to_excel(net,'net_drop_' + str(x) + '.xlsx')

In [ ]:
ppp.simple_plot(net)

In [ ]:
#Simple map of voltage levels of lines

ppp.plotly.vlevel_plotly(net,on_map='True')

#NOTE: The map generates correctly, but you may need to zoom out quite a bit before the network becomes visible.
#If you get error that lat/long are not valid, please check that you are loading 'net_geodata.xlsx'

In [ ]:
#Map of power flows (loading percentages)

pp.runpp(net)
ppp.plotly.pf_res_plotly(net,on_map='True')

#NOTE: The map generates correctly, but you may need to zoom out quite a bit before the network becomes visible.
#If you get error that lat/long are not valid, please check that you are loading 'net_geodata.xlsx'

In [ ]:
'''
This cell is for mitigation strategies.

It creates a new line between two specified buses and then re-runs the network with the new line.
'''

A = int(input("New line start at bus > "))
B = int(input("New line ends at bus > "))

#Operation performed twice for a double circuit line

pp.create_line_from_parameters(net, from_bus = A, to_bus = B, length_km=1, r_ohm_per_km = .32, x_ohm_per_km = 8.048, c_nf_per_km = 357.5, max_i_ka = 4.47)
pp.create_line_from_parameters(net, from_bus = A, to_bus = B, length_km=1, r_ohm_per_km = .32, x_ohm_per_km = 8.048, c_nf_per_km = 357.5, max_i_ka = 4.47)

pp.to_excel(net,'net_add_line.xlsx')